In [1]:
# data from FoodData Central: https://fdc.nal.usda.gov/
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# read subset of branded food csv
branded = pd.read_csv('./data_csvs/branded_food.csv', 
                      usecols=['fdc_id', 'brand_owner', 'brand_name', 'ingredients', 'branded_food_category'])

print('dataframe shape:', branded.shape)
branded.head()

dataframe shape: (1947155, 5)


,fdc_id,brand_owner,brand_name,ingredients,branded_food_category
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,Oils Edible
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",Herbs/Spices/Extracts
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",Prepared Soups
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",Prepared Soups
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",Herbs/Spices/Extracts


In [3]:
# read subset of food csv
food = pd.read_csv('./data_csvs/food.csv', usecols=['fdc_id', 'description'])
print('dataframe shape:', food.shape)
food.head()

dataframe shape: (1947155, 2)


,fdc_id,description
0,1105904,WESSON Vegetable Oil 1 GAL
1,1105905,SWANSON BROTH BEEF
2,1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER
3,1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI
4,1105908,SWANSON BROTH CHICKEN


In [4]:
# join food and branded dataframes
branded_food = branded.join(food.set_index('fdc_id'), on='fdc_id', how='left')
print('dataframe shape:', branded_food.shape)
branded_food.head()

dataframe shape: (1947155, 6)


,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,Oils Edible,WESSON Vegetable Oil 1 GAL
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",Herbs/Spices/Extracts,SWANSON BROTH BEEF
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",Herbs/Spices/Extracts,SWANSON BROTH CHICKEN


In [5]:
# clean up the column entries
# strip whitespace
for col in ['brand_owner', 'brand_name', 'description']:
    branded_food[col] = branded_food[col].str.strip()
    
# upper case column entries
for col in ['brand_owner', 'brand_name']:
    branded_food[col] = branded_food[col].str.upper()
    
# lower case column entries, clean up text
branded_food['ingredients'] = branded_food['ingredients'].str.lower()
branded_food['ingredients'] = branded_food['ingredients'].str.replace('ingredients:','').str.strip()

branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description
0,1105904,RICHARDSON OILSEED PRODUCTS (US) LIMITED,NaN,vegetable oil,Oils Edible,WESSON Vegetable Oil 1 GAL
1,1105905,CAMPBELL SOUP COMPANY,NaN,"beef stock, contains less than 2% of: mirepoix...",Herbs/Spices/Extracts,SWANSON BROTH BEEF
2,1105906,CAMPBELL SOUP COMPANY,NaN,"clam stock, potatoes, clams, cream, vegetable ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER
3,1105907,CAMPBELL SOUP COMPANY,NaN,"water, cream, broccoli, celery, vegetable oil ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI
4,1105908,CAMPBELL SOUP COMPANY,NaN,"chicken stock, contains less than 2% of: yeast...",Herbs/Spices/Extracts,SWANSON BROTH CHICKEN


In [6]:
# recategorize some food items that were miscategorized in original database entry

# bool indicating if row 'branded food category' is vegetable category
cond = [True if 'vegetable' in str(branded_food.at[idx, 'branded_food_category']).lower() else False 
        for idx in branded_food.index]
# get indexes of vegetable based categories
vegprod_idx = branded_food[cond].index

# iterate though descriptions to recategorize
for idx in vegprod_idx:
    # get description, covert to lower case
    fooddescript = str(branded_food.at[idx, 'description']).lower()
    # check for words that indicate different category
    if any(wrd in fooddescript for wrd in ['burger', 'patties', 'patty']):
        category = 'Meat Substitutes'
    elif 'pizza' in fooddescript:
        category = 'Pizza'
    elif any(wrd in fooddescript for wrd in ['diced', 'crushed']):
        category = 'Canned Vegetables'
    elif any(wrd in fooddescript for wrd in ['paste', 'pesto', 'puree', ' sauce']):
        category = 'sauces, condiments'
    elif 'snack' in fooddescript:
        category = 'Other Snacks'
    else:
        continue
        
    # fill in catgory    
    branded_food.at[idx, 'branded_food_category'] = category

# indexes of 'processed cereal products' branded food category
cond = branded_food['branded_food_category'] == 'Processed Cereal Products'
cerealprod_idx = branded_food[cond].index

# iterate though descriptions to recategorize
for idx in cerealprod_idx:
    # get description, covert to lower case
    fooddescript = str(branded_food.at[idx, 'description']).lower()
    # check for words that indicate different category
    if ' bar' in fooddescript:
        category = 'Cereal/Muesli Bars'
    else:
        continue
        
    # fill in catgory    
    branded_food.at[idx, 'branded_food_category'] = category

    
# rows with branded food category filled (not nan)
bfc_notnanidx = branded_food[branded_food['branded_food_category'].notna()].index

# iterate though descriptions to recategorize
for idx in bfc_notnanidx:
    # get description, covert to lower case
    fooddescript = str(branded_food.at[idx, 'description']).lower()
    
    if 'cadbury' in fooddescript:
        category = 'candy'
    elif "kellogg's all-in-one bowls/pouches" in fooddescript:
        category = 'cereal'
    elif 'yakult' in fooddescript:
        category = 'Yogurt'
    elif 'rimming salt' in fooddescript:
        category = 'sauces, condiments'
    else:
        continue
        
    # fill in catgory    
    branded_food.at[idx, 'branded_food_category'] = category

In [7]:
# check number of nans in joined dataframes
branded_food.isna().sum()

fdc_id                        0
brand_owner               16890
brand_name               547847
ingredients                5393
branded_food_category     10608
description                   1
dtype: int64

In [8]:
# rows with nan in branded food category
bfc_nanidx = branded_food[branded_food['branded_food_category'].isna()].index

In [9]:
# check rows with nan for branded food category
branded_food.loc[bfc_nanidx, :][50:80]
# show more characters in description
# with pd.option_context("display.max_colwidth", 100):
#     print(branded_food.loc[bfc_nanidx,'description'][50:80])

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description
33985,345569,CARGILL,NaN,"turkey, natural flavoring.",NaN,Honeysuckle White 93/7 Ground Turkey
33986,345570,CARGILL,NaN,"turkey breast, natural flavoring",NaN,Honeysuckle White 99% Ground Turkey Breast
34101,345685,RED GOLD,NaN,tomato paste and citric acid.,NaN,Red Gold 6oz. Tomato Paste
34102,345686,RED GOLD,NaN,"tomatoes with juice, water, chopped green chil...",NaN,Red Gold 10oz Diced Tomatoes w/ Green Chilies ...
34103,345687,RED GOLD,NaN,"tomatoes with juice, water, chopped green chil...",NaN,Red Gold 10oz Petite Diced Tomatoes w/ Green C...
34104,345688,RED GOLD,NaN,"tomato concentrate, high fructose corn syrup, ...",NaN,Red Gold 14oz Ketchup
34105,345689,RED GOLD,NaN,"tomato concentrate, high fructose corn syrup, ...",NaN,Red Gold 20oz Ketchup Upside Down Bottle
34106,345690,RED GOLD,NaN,"tomato concentrate, sugar, distilled vinegar, ...",NaN,Red Gold 20oz. Tomato Ketchup with Real Sugar
34107,345691,RED GOLD,NaN,"organic tomato concentrate (water, organic tom...",NaN,Red Gold 20oz. Organic Ketchup USDB
34108,345692,RED GOLD,NaN,"tomato concentrate from red ripe tomatoes, tap...",NaN,Red Gold 19.5oz. Naturally Balanced Ketchup


In [10]:
# define key words to search for in food description, use key words to fill nans in branded food category
candywords = ['candy', 'cadbury', 'chocolate bar']

snackwords = [' bar', 'bite', 'chips', 'crisps', ' cracker', 'dorito', 'granola', 'grahams', 'gummie', "lay's",
              'popcorn', 'pretzel', 'puff', 'seeds', 'slim jim', 'snack', 'skins', 'square', 'trail mix'] # not 'bar mix'
notsnackwords = ['bar mix', 'barq', 'cookie dough', 'popcorn chicken', 'boyardee', 'soup'] # , 'chocolate bar'

drinkwords = ['beverage', 'bottle', 'cola ', 'coke', 'drink', 'juice', 'punch', 'soda', 'sparkling', 'tea ', 'quencher']

cerealwords = ['cereal', 'instant oat'] # not 'cereal bar'

prepwords = ['bowl', 'burrito', 'chili', 'chowder', 'dinner', 'dumpling', 'fried', 'lasagna', 'linguini', ' meal', 
             'microwave', 'nugget', 'pizza', 'plate', 'pot pie', 'potsticker', 'ravioli', 'rigatoni', 'skillet', 
             'steamer', 'soup', 'tenders','toaster', 'tortellini']

saucewords = ['broth', 'dressing', 'gravy', 'paste', 'sauce', 'salsa', 'spread', 'topping', 'ketchup'] # 'peanut butter'

meatwords = ['beef', 'bologna', 'crab', 'fillet', 'flounder', 'frank', 'legs', 'link', 'loin', 
             'patty', 'pattie', 'pork', 'portion', 'ribeye', 'salami', 'sausage', 'turkey', 'tuna'] # , 'chicken'
            # brand owner 'beaver street fisheries'
    
vegwords = ['beans', 'berries', 'crushed', 'diced', 'peas', 'peeled']

dairywords = ['yogurt']

dessertwords = ['pie', 'cobbler']

In [11]:
%%time
# fill branded food category nans 
for idx in bfc_nanidx:
    # get description, covert to lower case
    fooddescript = str(branded_food.at[idx, 'description']).lower()
    
    # check food description for words that would likely categorize them
    # category - candy
    if any(wrd in fooddescript for wrd in candywords):
        category = 'candy'
    # category - snack foods
    elif any(wrd in fooddescript for wrd in snackwords) and all(wrd not in fooddescript for wrd in notsnackwords):
        category = 'snacks'
    # category - cereal
    elif any(wrd in fooddescript for wrd in cerealwords) and 'cereal bar' not in fooddescript:
        category = 'cereal'
    # category - prepared foods
    elif any(wrd in fooddescript for wrd in prepwords) and 'chilie' not in fooddescript:
        category = 'prepared food/meals'
    # category - sauces and condiments
    elif any(wrd in fooddescript for wrd in saucewords):
        category = 'sauces, condiments'
    # category - drinks
    elif any(wrd in fooddescript for wrd in drinkwords) and all(wrd not in fooddescript for wrd in vegwords):
        category = 'drinks'
    # category - meat, poultry, seafood
    elif 'beaver street fisheries' in str(branded_food.at[idx, 'brand_owner']).lower() or \
    any(wrd in fooddescript for wrd in meatwords):
        category = 'meat, poultry, seafood'
    # category - dairy
    elif any(wrd in fooddescript for wrd in dairywords):
        category = 'dairy'
    # category - vegetables and fruits
    elif any(wrd in fooddescript for wrd in vegwords):
        category = 'veg, fruit'
    # category - desserts
    elif any(wrd in fooddescript for wrd in dessertwords):
        category = 'dessert'
    # otherwise leave nan
    else:
        category = np.nan
        
    # fill in catgory    
    branded_food.at[idx, 'branded_food_category'] = category

CPU times: total: 1.39 s
Wall time: 1.79 s


In [12]:
# check nans of joined dataframes
branded_food.isna().sum()

fdc_id                        0
brand_owner               16890
brand_name               547847
ingredients                5393
branded_food_category      2846
description                   1
dtype: int64

In [13]:
# check filled values of branded food category
branded_food.loc[bfc_nanidx,:][150:180]

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description
34444,346028,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), less than ...",NaN,"Hunt's Tomato Puree, 29 oz, 29 OZ"
34445,346029,CONAGRA BRANDS,NaN,"tomato paste, citric acid","sauces, condiments","HUNTS Tomato Paste, 29 OZ"
34446,346030,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), water, les...","sauces, condiments","HUNTS No Salt Added Tomato Sauce, 8 OZ"
34447,346031,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), water, les...","sauces, condiments","HUNTS Tomato Sauce, 15 OZ"
34448,346032,CONAGRA BRANDS,NaN,"vine-ripened tomatoes, tomato juice, less than...","veg, fruit","ANGELA MIA Petite Diced Tomatoes, #10 Can, 6/1..."
34449,346033,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), water, les...","sauces, condiments","HUNTS Tomato Sauce, #10 Can, 6/105 oz., 105 OZ"
34450,346034,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), water, les...","sauces, condiments","Hunt's Tomato Sauce with Basil, Garlic, and Or..."
34451,346035,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), water, les...","sauces, condiments","HUNTS Tomato Sauce, 8 OZ"
34452,346036,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), tomatoes, ...","sauces, condiments","ANGELA MIA No Salt Added Spaghetti Sauce, Pouc..."
34453,346037,CONAGRA BRANDS,NaN,"tomato puree (water, tomato paste), tomatoes, ...","sauces, condiments","ANGELA MIA Marinara Sauce, #10 Can, 6/104 oz.,..."


In [14]:
branded_food['branded_food_category'].value_counts()

branded_food_category
Popcorn, Peanuts, Seeds & Related Snacks                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           92023
Candy                                                                                                                                                                                                                                                                                                                                    

In [15]:
# condense 'branded food category' values of interest into a smaller number of more general categories
# load csv with 'branded food category' values condensed into 7 overall categories
catdf = pd.read_csv('./category-sorting-files/FoodData Project - Branded Food Categories.csv')
catdf.head()

,"MEAT, POULTRY, SEAFOOD, MEAT SUBSTITUTES","FRUITS, VEGETABLES, LEGUMES","EGGS, DAIRY, DAIRY SUBSTITUTES",DRINKS (NON-DAIRY),CEREAL,SNACKS,PREPARED FOODS/MEALS
0,Frozen Fish & Seafood,Pre-Packaged Fruit & Vegetables,Cheese,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",Cereal,"Popcorn, Peanuts, Seeds & Related Snacks",Frozen Dinners & Entrees
1,Other Meats,Frozen Vegetables,Milk,Water,Processed Cereal Products,"Chips, Pretzels & Snacks,",Frozen Appetizers & Hors D'oeuvres
2,Canned Seafood,Canned Vegetables,Plant Based Milk,Other Drinks,Cereals Products - Ready to Eat (Shelf Stable),"Snack, Energy & Granola Bars",Pizza
3,Fish & Seafood,Canned Fruit,Ice Cream & Frozen Yogurt,Soda,Breakfast Cereals - Hot And Cold,Other Snacks,Other Soups
4,Canned Tuna,Canned & Bottled Beans,Cheese/Cheese Substitutes,Iced & Bottle Tea,Cereal Products - Ready to Eat (Perishable),Wholesome Snacks,Canned Soup


In [16]:
# create new 'category' column with foods recategorized into the new categories
# iterate through each column, column label will be new category value
for category in catdf.columns:
    # iterate through 'branded food category' values in each column
    for bfc in catdf[category]:
        # create mask for branded food category with value bfc
        mask = branded_food['branded_food_category'] == bfc
        # fill category column with new category
        branded_food.loc[mask, 'category'] = category
        
branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description,category
0,1105904,RICHARDSON OILSEED PRODUCTS (US) LIMITED,NaN,vegetable oil,Oils Edible,WESSON Vegetable Oil 1 GAL,NaN
1,1105905,CAMPBELL SOUP COMPANY,NaN,"beef stock, contains less than 2% of: mirepoix...",Herbs/Spices/Extracts,SWANSON BROTH BEEF,NaN
2,1105906,CAMPBELL SOUP COMPANY,NaN,"clam stock, potatoes, clams, cream, vegetable ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,PREPARED FOODS/MEALS
3,1105907,CAMPBELL SOUP COMPANY,NaN,"water, cream, broccoli, celery, vegetable oil ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,PREPARED FOODS/MEALS
4,1105908,CAMPBELL SOUP COMPANY,NaN,"chicken stock, contains less than 2% of: yeast...",Herbs/Spices/Extracts,SWANSON BROTH CHICKEN,NaN


In [17]:
# drop food categories that are not of interest (ie. candy, sauces, baking mix...)
branded_food.dropna(axis=0, subset='category', inplace=True)

print('branded food shape', branded_food.shape)
branded_food.head()

branded food shape (1202625, 7)


,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description,category
2,1105906,CAMPBELL SOUP COMPANY,NaN,"clam stock, potatoes, clams, cream, vegetable ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,PREPARED FOODS/MEALS
3,1105907,CAMPBELL SOUP COMPANY,NaN,"water, cream, broccoli, celery, vegetable oil ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,PREPARED FOODS/MEALS
5,1105909,CAMPBELL SOUP COMPANY,NaN,"water, pea beans, carrots, cooked ham - water ...",Prepared Soups,CAMPBELL'S SOUP BEAN AND HAM,PREPARED FOODS/MEALS
8,1105912,CAMPBELL SOUP COMPANY,NaN,"tomato puree (water, tomato paste), water, hig...",Prepared Soups,CAMPBELL'S SOUP TOMATO,PREPARED FOODS/MEALS
9,1105913,CAMPBELL SOUP COMPANY,NaN,"water, tomato puree (water, tomato paste), chi...",Dough Based Products / Meals,CAMPBELL'S PASTA SPAGHETTIOS W FRANKS,PREPARED FOODS/MEALS


In [18]:
branded_food['category'].value_counts()

category
SNACKS                                      330989
EGGS, DAIRY, DAIRY SUBSTITUTES              224513
PREPARED FOODS/MEALS                        178116
DRINKS (NON-DAIRY)                          161215
MEAT, POULTRY, SEAFOOD, MEAT SUBSTITUTES    140145
FRUITS, VEGETABLES, LEGUMES                 125490
CEREAL                                       42157
Name: count, dtype: int64

In [19]:
descriptcount = branded_food['description'].value_counts()
descriptcount[descriptcount > 1][:10]

description
ICE CREAM              2451
2% REDUCED FAT MILK    1909
CUT GREEN BEANS        1517
SODA                   1309
PREMIUM ICE CREAM      1192
FAT FREE MILK          1167
PINTO BEANS            1135
WHOLE MILK             1086
1% LOWFAT MILK          999
DICED TOMATOES          993
Name: count, dtype: int64

In [20]:
# create subcategories
# load csv with some 'branded food category' values put into subcategories
subcatdf = pd.read_csv('./category-sorting-files/FoodData Project - Branded Food SubCategories.csv')
subcatdf.head()

,SEAFOOD,UNPROCESSED MEATS,PROCESSED MEATS,"PREPARED FRUITS, VEGETABLES","UNPREPARED FRUITS, VEGETABLES",CHEESE,ICE CREAM,MILK,YOGURT,EGGS,...,"NUTS, SEEDS","COOKIES, CRACKERS","CHIPS, MUNCHIES",PIZZA,"SOUP, STEW",DELI,BREAKFAST,DINNER,APPETIZER,SIDES
0,Frozen Fish & Seafood,"Poultry, Chicken & Turkey",Chicken - Prepared/Processed,Frozen Vegetables,Vegetables Unprepared/Unprocessed (Frozen),Cheese,Ice Cream & Frozen Yogurt,Milk,Yogurt,Eggs & Egg Substitutes,...,"Popcorn, Peanuts, Seeds & Related Snacks",Cookies & Biscuits,"Chips, Pretzels & Snacks,",Pizza,Other Soups,Deli Salads,"Frozen Breakfast Sandwiches, Biscuits & Meals",Frozen Dinners & Entrees,Frozen Appetizers & Hors D'oeuvres,Grain Based Products / Meals
1,Canned Seafood,"Frozen Poultry, Chicken & Turkey",Frozen Chicken - Processed,Canned Vegetables,Fruits Unprepared/Unprocessed (Shelf Stable),Cheese/Cheese Substitutes,Ice Cream/Ice Novelties (Shelf Stable),Plant Based Milk,Yogurt/Yogurt Substitutes,Eggs/Eggs Substitutes,...,Fruit/Nuts/Seeds Combination,Flavored Snack Crackers,Other Snacks,Pies/Pastries/Pizzas/Quiches - Savoury (Frozen),Canned Soup,Prepared Subs & Sandwiches,"Frozen Pancakes, Waffles, French Toast & Crepes",Pasta Dinners,Frozen Meals,Grain Based Products / Meals - Not Ready to Ea...
2,Fish & Seafood,Frozen Chicken - Portions,Turkey - Prepared/Processed,Canned Fruit,Fruits Unprepared/Unprocessed (Frozen),Cheese - Speciality,Ice-Cream Take Home,Milk/Milk Substitutes,Yogurt (Perishable),Eggs,...,Popcorn (Shelf Stable),Crackers & Biscotti,Wholesome Snacks,NaN,Prepared Soups,Prepared Wraps and Burittos,"Breakfast Sandwiches, Biscuits & Meals",Pizza Mixes & Other Dry Dinners,Frozen Foods,Frozen Prepared Sides
3,Canned Tuna,Turkey - Unprepared/Unprocessed,Fresh Chicken - Processed,Canned & Bottled Beans,Vegetables – Unprepared/Unprocessed (Frozen),Cheese - Block,Ice-Cream/Block Single,Milk/Cream - Shelf Stable,Dairy Foods/Yoghurts,Eggs Products/Substitutes,...,Snack Foods - Nuts,Biscuits/Cookies,Snack Foods - Chips,NaN,Soups - Prepared (Shelf Stable),Sandwiches/Filled Rolls/Wraps,Breakfast Foods,Prepared Meals,NaN,Flavored Rice Dishes
4,Shellfish Unprepared/Unprocessed,Meat/Poultry/Other Animals Unprepared/Unproce...,Meat/Poultry/Other Animals – Prepared/Processed,Tomatoes,Vegetables – Unprepared/Unprocessed (Shelf Sta...,Processed Cheese & Cheese Novelties,Ice-Cream/Ice-Block Multi Pack,Cream/Cream Substitutes,Yogurt/Yogurt Substitutes (Perishable),Dairy/Egg Based Products / Meals,...,Nuts/Seeds Prepared/Processed,Biscuits/Cookies (Shelf Stable),Chips/Crisps/Snack Mixes - Natural/Extruded (S...,NaN,Chili & Stew,Sandwiches/Filled Rolls/Wraps (Frozen),"Pancakes, Waffles, French Toast & Crepes",Vegetable Based Products / Meals - Not Ready t...,NaN,Vegetable Based Products / Meals - Not Ready t...


In [21]:
# create 'subcategory' column, partially fill using 'branded food category'
# iterate through each column of subcategory dataframe, column label will be new subcategory value
for subcategory in subcatdf.columns:
    # iterate through 'branded food category' values in each column
    for bfc in subcatdf[subcategory]:
        # create mask for branded food category with value bfc
        mask = branded_food['branded_food_category'] == bfc
        # fill subcategory column with new subcategory
        branded_food.loc[mask, 'subcategory'] = subcategory

In [22]:
# check subcategory nan values
branded_food[branded_food['subcategory'].isna()] # [200:250]

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description,category,subcategory
50,1105954,WELCH FOODS INC.,NaN,"orange juice from concentrate (filtered water,...",Non Alcoholic Beverages Ready to Drink,10 FL OZ 100% JUICE - ORANGE,DRINKS (NON-DAIRY),NaN
51,1105955,WELCH FOODS INC.,NaN,"filtered water, high fructose corn syrup, appl...",Non Alcoholic Beverages Ready to Drink,16 FL OZ JUICE DRINK - FRUIT PUNCH,DRINKS (NON-DAIRY),NaN
52,1105956,WELCH FOODS INC.,NaN,"filtered water, high fructose corn syrup, cran...",Non Alcoholic Beverages Ready to Drink,11.5 FL OZ JUICE DRINK - APPLE CRANBERRY,DRINKS (NON-DAIRY),NaN
53,1105957,WELCH FOODS INC.,NaN,"orange juice from concentrate (filtered water,...",Non Alcoholic Beverages Ready to Drink,5.5 FL OZ 100% JUICE - ORANGE,DRINKS (NON-DAIRY),NaN
54,1105958,WELCH FOODS INC.,NaN,"filtered water, high fructose corn syrup, cran...",Non Alcoholic Beverages Ready to Drink,5.5 FL OZ JUICE COCKTAIL - CRANBERRY,DRINKS (NON-DAIRY),NaN
...,...,...,...,...,...,...,...,...
1947077,2662191,TARGET STORES,ARCHER FARMS,"water, sugar, coffee, natural flavors.",Other Drinks,"CARAMEL MACCHIATO COFFEE CONCENTRATE, CARAMEL ...",DRINKS (NON-DAIRY),NaN
1947078,2662192,TARGET STORES,ARCHER FARMS,"water, sugar, coffee.",Other Drinks,"DONUT SHOP COFFEE CONCENTRATE, DONUT SHOP",DRINKS (NON-DAIRY),NaN
1947079,2662193,TARGET STORES,ARCHER FARMS,"water, coffee, natural flavors, sucralose.",Other Drinks,CARAMEL MACCHIATO SUGAR FREE COFFEE CONCENTRAT...,DRINKS (NON-DAIRY),NaN
1947141,2662255,TARGET STORES,ARCHER FARMS,"maple crunch granola (whole grain rolled oats,...",Cereal,"MAPLE CRUNCH GRANOLA, MAPLE CRUNCH",CEREAL,NaN


In [23]:
# indexes where subcategory is nan
subcat_nanidx = branded_food[branded_food['subcategory'].isna()].index

In [24]:
# define key words to search for in food description, use key words to fill nans in subcategory
juicewords = ['juice', 'v8']

sodawords = ['bottle', 'cola ', 'coke', 'soda']

teacofwords = ['tea', 'kombucha', 'coffee', 'latte']

sportwords = ['gatorade', 'quencher']

waterwords = ['sparkling']

barwords = [' bar']

oatwords = [' oat', 'oatmeal']

soupwords = ['soup', 'campbell']

In [25]:
%%time
# fill subcategory for items not well categorized by the branded food category
# fill subcategory nans 
for idx in subcat_nanidx:
    # get description, covert to lower case
    fooddescript = str(branded_food.at[idx, 'description']).lower()
    foodcat = branded_food.at[idx, 'category']
    
    # check food description for words that would likely categorize them
    # category - drinks
    if foodcat == 'DRINKS (NON-DAIRY)':
        if any(wrd in fooddescript for wrd in juicewords):
            subcategory = 'JUICE'
        elif any(wrd in fooddescript for wrd in sodawords):
            subcategory = 'SODA'
        elif any(wrd in fooddescript for wrd in teacofwords):
            subcategory = 'TEA, COFFEE'
        elif any(wrd in fooddescript for wrd in sportwords):
            subcategory = 'SPORT, ENERGY DRINKS'
        elif any(wrd in fooddescript for wrd in waterwords):
            subcategory = 'WATER'
        else: 
            subcategory = 'OTHER'
            
    # category - snack foods
    elif foodcat == 'SNACKS':
        if any(wrd in fooddescript for wrd in barwords):
            subcategory = 'SNACK BARS'
        else: 
            subcategory = 'OTHER'
            
    # category - prepared foods
    elif foodcat == 'PREPARED FOODS/MEALS':
        if any(wrd in fooddescript for wrd in soupwords):
            subcategory = 'SOUP, STEW'
        else: 
            subcategory = 'OTHER'
            
    # category - cereal
    elif foodcat == 'CEREAL':
        if any(wrd in fooddescript for wrd in oatwords):
            subcategory = 'OATMEAL'
        elif 'granola' in fooddescript:
            subcategory = 'GRANOLA'
        else: 
            subcategory = 'OTHER'
    # otherwise leave nan
    else:
        subcategory = np.nan
        
    # fill in subcatgory    
    branded_food.at[idx, 'subcategory'] = subcategory

CPU times: total: 22.7 s
Wall time: 27.7 s


In [26]:
branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,branded_food_category,description,category,subcategory
2,1105906,CAMPBELL SOUP COMPANY,NaN,"clam stock, potatoes, clams, cream, vegetable ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,PREPARED FOODS/MEALS,"SOUP, STEW"
3,1105907,CAMPBELL SOUP COMPANY,NaN,"water, cream, broccoli, celery, vegetable oil ...",Prepared Soups,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,PREPARED FOODS/MEALS,"SOUP, STEW"
5,1105909,CAMPBELL SOUP COMPANY,NaN,"water, pea beans, carrots, cooked ham - water ...",Prepared Soups,CAMPBELL'S SOUP BEAN AND HAM,PREPARED FOODS/MEALS,"SOUP, STEW"
8,1105912,CAMPBELL SOUP COMPANY,NaN,"tomato puree (water, tomato paste), water, hig...",Prepared Soups,CAMPBELL'S SOUP TOMATO,PREPARED FOODS/MEALS,"SOUP, STEW"
9,1105913,CAMPBELL SOUP COMPANY,NaN,"water, tomato puree (water, tomato paste), chi...",Dough Based Products / Meals,CAMPBELL'S PASTA SPAGHETTIOS W FRANKS,PREPARED FOODS/MEALS,DINNER


In [27]:
# need to check nutrition values of duplicate food products once dataframes combined
branded_food.to_csv('cleaned_branded_food.csv', index=False)